In [ ]:
#!pip install accelerate

In [1]:
import dspy

In [2]:
lm = dspy.HFModel(model="google/flan-t5-base", )
retriever = dspy.ColBERTv2(url='http://localhost:8001/api/search')
dspy.settings.configure(lm=lm, rm=retriever)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
class BasicQA(dspy.Signature):
    """Answer the question with a short answer."""

    question = dspy.InputField()
    answer = dspy.OutputField()

qa = dspy.Predict(BasicQA)

In [4]:
ans = qa(question="What is Fringe?")
print(ans.answer)

/miniconda/envs/easyrag/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


music


In [5]:
ans = qa(question="What is the name of the third episode of Fringe?")
print(ans.answer)

"Fringe" ---


In [7]:
class GenerateAnswerWithContext(dspy.Signature):
    """Answer the question with a short answer."""

    context = dspy.InputField(desc="may contain relevant facts to consider")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="short answer")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswerWithContext)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)
    
rag = RAG()

In [8]:
ans = rag(question="What is Fringe?")
print(ans.answer)

an American science fiction television series created by J. J. Abrams, Alex Kurtzman, and Roberto Orci.


In [9]:
ans = rag(question="What is the name of the third episode of Fringe?")
print(ans.answer)

The Ghost Network is the third episode of the first season of the American science fiction drama television series Fringe. So, the final answer is The Ghost Network.


In [10]:
ans = rag(question="What does Fringe Division do?")
print(ans.answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (846 > 512). Running this sequence through the model will result in indexing errors


The Fringe Division investigates cases relating to fringe science. So, the final answer is investigates cases relating to fringe science.


In [11]:
ans = rag(question="Which episodes are directed by J. J. Abrams?")
print(ans.answer)

[1] """The Arrival"" is the fourth episode of the first season of the American science fiction drama television series Fringe. The episode was written by the series co-creator and executive producer J. J. Abrams and executive producer Jeff Pinkner. Paul A. Edwards directed the episode, his second of the season. Early


In [12]:
lm.inspect_history(n=5)





Answer the question with a short answer.

---

Follow the following format.

Context: may contain relevant facts to consider

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: short answer

---

Context:
[1] «"""The Ghost Network"" is the third episode of the first season of the American science fiction drama television series Fringe. The episode was written by co-executive producer David H. Goodman and supervising producer J. R. Orci, and was directed by Frederick E. O. Toye. The episode follows the Fringe team's investigation into a bus that was filled with amber, encasing the people inside. They discover a man named Roy (Zak Orth) who predicted it and other similar events, and Walter realizes Roy has connections to a past experiment he did over twenty years ago, called the ""Ghost Network""."»
[2] «"""The Arrival"" is the fourth episode of the first season of the American science fiction drama television series Fringe. 